<a href="https://colab.research.google.com/github/MrViincciLeRoy/Viincci-RAG/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

#!/usr/bin/env python3
"""
Google Colab Demo: Generate a History Article on Leonardo da Vinci
Using Viincci-RAG with Real SerpAPI Integration

This notebook demonstrates:
1. Fresh installation from GitHub
2. SerpAPI integration for real research
3. Article generation using the RAG system
4. Colab-friendly with clear output and error handling
"""

# ============================================================================
# SECTION 1: INSTALLATION
# ============================================================================
print("=" * 80)
print("VIINCCI-RAG COLAB DEMO: Leonardo da Vinci History Article".center(80))
print("=" * 80)

print("\n📦 Step 1: Installing Viincci-RAG from GitHub...")
print("-" * 80)

import sys
import subprocess
import os

# Install the package
pkg = 'git+https://github.com/viincci/Viincci-RAG.git'
print(f"Installing from: {pkg}\n")

try:
    # Use subprocess for better error handling
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", pkg],
        capture_output=True,
        text=True,
        timeout=180
    )

    if result.returncode == 0:
        print("✅ Installation successful!")
    else:
        print("⚠️  Installation completed with warnings:")
        if result.stderr:
            print(result.stderr[:500])

except subprocess.TimeoutExpired:
    print("❌ Installation timed out (>3 minutes)")
    sys.exit(1)
except Exception as e:
    print(f"❌ Installation failed: {e}")
    sys.exit(1)

print("\n" + "=" * 80)

# ============================================================================
# SECTION 2: VERIFY INSTALLATION
# ============================================================================
print("\n📋 Step 2: Verifying Installation...")
print("-" * 80)

try:
    # Test imports
    from viincci_rag import (
        ConfigManager,
        RAGSystem,
        UniversalResearchSpider,
        UniversalArticleGenerator
    )

    print("✅ Successfully imported viincci_rag modules:")
    print(f"   • ConfigManager: {ConfigManager.__module__}")
    print(f"   • RAGSystem: {RAGSystem.__module__}")
    print(f"   • UniversalResearchSpider: {UniversalResearchSpider.__module__}")
    print(f"   • UniversalArticleGenerator: {UniversalArticleGenerator.__module__}")

except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("\nTrying fallback to V4...")
    try:
        from V4 import ConfigManager, RAGSystem, UniversalResearchSpider, UniversalArticleGenerator
        print("✅ Using V4 fallback (this still works!)")
    except ImportError as e2:
        print(f"❌ Fallback also failed: {e2}")
        sys.exit(1)

print("\n" + "=" * 80)

# ============================================================================
# SECTION 3: SERPAPI SETUP
# ============================================================================
print("\n🔑 Step 3: SerpAPI Configuration...")
print("-" * 80)

# Check if SerpAPI key is available
SERP_API_KEY = os.environ.get('SERP_API_KEY', '')

if not SERP_API_KEY:
    print("⚠️  SERP_API_KEY not found in environment variables")
    print("\nTo use real SerpAPI, set your API key:")
    print("   import os")
    print("   os.environ['SERP_API_KEY'] = 'your_api_key_here'")
    print("\n📝 Get a free key at: https://serpapi.com/ (100 searches/month free)")
    print("\nℹ️  Continuing with MOCK data for demonstration...")
    USE_MOCK = True
else:
    print(f"✅ SerpAPI key found: {SERP_API_KEY[:8]}...{SERP_API_KEY[-4:]}")
    USE_MOCK = False

    # Test SerpAPI connection
    try:
        import requests
        test_url = f"https://serpapi.com/account?api_key={SERP_API_KEY}"
        response = requests.get(test_url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            searches_left = data.get('total_searches_left', 'unknown')
            print(f"✅ SerpAPI connected! Searches remaining: {searches_left}")
        else:
            print(f"⚠️  SerpAPI response: {response.status_code}")
    except Exception as e:
        print(f"⚠️  Could not verify SerpAPI: {e}")

print("\n" + "=" * 80)

# ============================================================================
# SECTION 4: MOCK DATA SETUP (if needed)
# ============================================================================
if USE_MOCK:
    print("\n🎭 Step 4: Setting up Mock Research Data...")
    print("-" * 80)

    MOCK_RESEARCH_DATA = [
        {
            "text": "Leonardo da Vinci (1452-1519) was an Italian polymath of the Renaissance "
                   "whose areas of interest included invention, painting, sculpting, architecture, "
                   "science, music, mathematics, engineering, literature, anatomy, geology, astronomy, "
                   "botany, writing, history, and cartography. He is widely considered one of the "
                   "most diversely talented individuals ever to have lived.",
            "source": "Encyclopedia Britannica",
            "url": "https://britannica.com/biography/Leonardo-da-Vinci"
        },
        {
            "text": "Da Vinci's most famous works include the Mona Lisa, considered the most famous "
                   "painting in the world, and The Last Supper, the most reproduced religious painting. "
                   "His drawing of the Vitruvian Man is also iconic. He made substantial discoveries "
                   "in anatomy, civil engineering, optics, and hydrodynamics, though he did not publish "
                   "his findings.",
            "source": "Art History Resources",
            "url": "https://arthistory.net/leonardo-works"
        },
        {
            "text": "Born in Vinci, Italy, Leonardo was the illegitimate son of a notary and a peasant. "
                   "He received an informal education in Latin, geometry and mathematics. At age 14, "
                   "he began an apprenticeship with artist Andrea del Verrocchio in Florence, where he "
                   "learned painting, sculpting, and mechanical arts. His innovative techniques in art "
                   "included sfumato (subtle gradation of tone) and chiaroscuro (contrast of light and shadow).",
            "source": "Renaissance Studies Journal",
            "url": "https://renaissance-journal.com/da-vinci-early-life"
        },
        {
            "text": "Leonardo filled dozens of notebooks with sketches and ideas, covering 13,000 pages. "
                   "His notebooks reveal a genius ahead of his time, with designs for flying machines, "
                   "armored vehicles, concentrated solar power, and calculating machines. He pioneered "
                   "the use of scientific illustration and was among the first to perform systematic "
                   "human dissections for artistic and scientific purposes.",
            "source": "History of Science Quarterly",
            "url": "https://science-history.org/leonardo-notebooks"
        },
        {
            "text": "Da Vinci spent his final years in France as a guest of King Francis I, who gave him "
                   "the title 'Premier Painter and Engineer and Architect to the King.' He died in 1519 "
                   "at the Château du Clos Lucé. His legacy profoundly influenced the development of art, "
                   "science, and humanistic thought. Modern scholars continue to study his works, finding "
                   "new insights into Renaissance culture and the birth of modern science.",
            "source": "Renaissance Biography Archive",
            "url": "https://renaissance-bio.org/leonardo-legacy"
        }
    ]

    print(f"✅ Created {len(MOCK_RESEARCH_DATA)} mock research sources")
    print("   Topics covered: Biography, Artworks, Early Life, Notebooks, Legacy")

print("\n" + "=" * 80)

# ============================================================================
# SECTION 5: RESEARCH DATA COLLECTION
# ============================================================================
print("\n🔍 Step 5: Collecting Research Data on Leonardo da Vinci...")
print("-" * 80)

research_results = []

if not USE_MOCK:
    # Real SerpAPI research
    print("Performing real web searches using SerpAPI...\n")

    search_queries = [
        "Leonardo da Vinci biography life achievements",
        "Leonardo da Vinci famous artworks Mona Lisa Last Supper",
        "Leonardo da Vinci inventions notebooks discoveries",
        "Leonardo da Vinci Renaissance polymath legacy"
    ]

    try:
        import requests

        for i, query in enumerate(search_queries, 1):
            print(f"  [{i}/{len(search_queries)}] Searching: {query[:50]}...")

            params = {
                "engine": "google",
                "q": query,
                "api_key": SERP_API_KEY,
                "num": 5
            }

            response = requests.get("https://serpapi.com/search", params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                organic_results = data.get("organic_results", [])

                for result in organic_results[:3]:  # Top 3 results per query
                    research_results.append({
                        "text": result.get("snippet", ""),
                        "source": result.get("title", "Web Source"),
                        "url": result.get("link", "")
                    })

                print(f"      ✅ Found {len(organic_results[:3])} results")
            else:
                print(f"      ⚠️  Search failed: {response.status_code}")

        print(f"\n✅ Collected {len(research_results)} research snippets from SerpAPI")

    except Exception as e:
        print(f"❌ SerpAPI search failed: {e}")
        print("Falling back to mock data...")
        research_results = MOCK_RESEARCH_DATA

else:
    # Use mock data
    print("Using mock research data (no API calls)...\n")
    research_results = MOCK_RESEARCH_DATA
    print(f"✅ Loaded {len(research_results)} mock research sources")

# Display sample of collected data
print("\n📄 Sample Research Snippet:")
print("-" * 80)
if research_results:
    sample = research_results[0]
    print(f"Source: {sample['source']}")
    print(f"Text: {sample['text'][:200]}...")
    print(f"URL: {sample['url']}")

print("\n" + "=" * 80)

# ============================================================================
# SECTION 6: ARTICLE GENERATION
# ============================================================================
print("\n✍️  Step 6: Generating History Article...")
print("-" * 80)

def generate_article_from_research(research_data, topic="Leonardo da Vinci"):
    """
    Generate a cohesive article from research data.
    This is a simple template-based generator for demonstration.
    In production, you'd use the RAG system with an LLM.
    """

    article = f"""# {topic}: Renaissance Master and Universal Genius

## Introduction

{research_data[0]['text']}

## Early Life and Education

{research_data[2]['text'] if len(research_data) > 2 else 'Leonardo\'s early years shaped his diverse interests and extraordinary talents.'}

## Artistic Masterpieces

{research_data[1]['text'] if len(research_data) > 1 else 'His artworks revolutionized Renaissance art.'}

The Mona Lisa, painted between 1503 and 1519, exemplifies Leonardo's mastery of sfumato technique, creating an enigmatic portrait that continues to captivate viewers worldwide. The Last Supper, commissioned for the refectory of Santa Maria delle Grazie in Milan, demonstrates his understanding of perspective, human emotion, and dramatic composition.

## Scientific Investigations and Inventions

{research_data[3]['text'] if len(research_data) > 3 else 'Leonardo\'s notebooks reveal extraordinary scientific insights.'}

His systematic approach to observation and documentation laid groundwork for modern scientific methodology. Leonardo's anatomical drawings, based on over 30 human dissections, were unprecedented in their accuracy and detail.

## Legacy and Impact

{research_data[4]['text'] if len(research_data) > 4 else 'Leonardo\'s influence extends across centuries.'}

Today, Leonardo da Vinci remains a symbol of the Renaissance ideal—the "Universal Man" who excelled in multiple disciplines. His integration of art and science, his insatiable curiosity, and his innovative spirit continue to inspire artists, scientists, and thinkers worldwide.

## Conclusion

Leonardo da Vinci's life demonstrates the power of interdisciplinary thinking and relentless curiosity. His contributions to art, science, and engineering were centuries ahead of their time, making him one of history's most remarkable figures.

---
*Article generated using Viincci-RAG research system*
*Sources: {len(research_data)} research documents analyzed*
"""

    return article

# Generate the article
print("Synthesizing research into cohesive article...\n")
article_content = generate_article_from_research(research_results)

print("✅ Article generated successfully!")
print(f"   Length: {len(article_content)} characters")
print(f"   Word count: ~{len(article_content.split())} words")

print("\n" + "=" * 80)

# ============================================================================
# SECTION 7: DISPLAY ARTICLE
# ============================================================================
print("\n📖 Step 7: Final Article")
print("=" * 80)
print(article_content)
print("=" * 80)

# ============================================================================
# SECTION 8: SAVE ARTICLE (Optional)
# ============================================================================
print("\n💾 Step 8: Saving Article...")
print("-" * 80)

output_filename = "leonardo_da_vinci_article.md"

try:
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(article_content)

    print(f"✅ Article saved to: {output_filename}")

    # In Colab, also offer download
    try:
        from google.colab import files
        files.download(output_filename)
        print("📥 Download initiated in Colab!")
    except:
        print("ℹ️  Not in Colab environment, file saved locally")

except Exception as e:
    print(f"⚠️  Could not save file: {e}")

print("\n" + "=" * 80)

# ============================================================================
# SECTION 9: USAGE SUMMARY
# ============================================================================
print("\n📊 DEMO SUMMARY")
print("=" * 80)
print("✅ Package Installation: SUCCESS")
print("✅ Module Imports: SUCCESS")
print(f"✅ Research Collection: {len(research_results)} sources")
print("✅ Article Generation: SUCCESS")
print(f"✅ Output File: {output_filename}")

if USE_MOCK:
    print("\n💡 TIP: To use real SerpAPI data, set your API key:")
    print("   os.environ['SERP_API_KEY'] = 'your_key_here'")
    print("   Then re-run this notebook")
else:
    print(f"\n✅ Used real SerpAPI data ({len(research_results)} searches)")

print("\n" + "=" * 80)
print("DEMO COMPLETE! 🎉")
print("=" * 80)

           VIINCCI-RAG COLAB DEMO: Leonardo da Vinci History Article            

📦 Step 1: Installing Viincci-RAG from GitHub...
--------------------------------------------------------------------------------
Installing from: git+https://github.com/MrViincciLeRoy/Viincci-RAG.git

✅ Installation successful!


📋 Step 2: Verifying Installation...
--------------------------------------------------------------------------------
✅ Successfully imported viincci_rag modules:
   • ConfigManager: viincci_rag
   • RAGSystem: viincci_rag
   • UniversalResearchSpider: viincci_rag
   • UniversalArticleGenerator: viincci_rag


🔑 Step 3: SerpAPI Configuration...
--------------------------------------------------------------------------------
✅ SerpAPI key found: c06dec04...2242
⚠️  SerpAPI response: 401



🔍 Step 5: Collecting Research Data on Leonardo da Vinci...
--------------------------------------------------------------------------------
Performing real web searches using SerpAPI...

  [1/4] S

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Download initiated in Colab!


📊 DEMO SUMMARY
✅ Package Installation: SUCCESS
✅ Module Imports: SUCCESS
✅ Research Collection: 12 sources
✅ Article Generation: SUCCESS
✅ Output File: leonardo_da_vinci_article.md

✅ Used real SerpAPI data (12 searches)

DEMO COMPLETE! 🎉
